## Get events 

In [1]:
import wget

!rm *.ndk

url = 'http://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/jan76_dec13.ndk'
wget.download( url )

years = range(14,18)
months  = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
for year in years:
    for month in months:
        url='http://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/NEW_MONTHLY/20%02d/%s%02d.ndk' % (year, month, year)
        #print(url)
        wget.download( url )

In [2]:
import obspy
from obspy.geodetics.base import gps2dist_azimuth
from obspy.clients.syngine import Client
from obspy.taup import TauPyModel

#cat = obspy.read_events('jan76_dec13.ndk')

cat = obspy.core.event.Catalog()

for year in years:
    for month in months:
        tmp = obspy.read_events('%s%02d.ndk' % (month, year))
        cat = cat + tmp.filter('magnitude >= 5.5')
    
print(cat)
    

1614 Event(s) in Catalog:
2014-01-01T16:03:34.800000Z | -13.890, +167.100 | 6.53 Mwc
2014-01-04T00:11:55.400000Z | -20.600,  -70.980 | 5.73 Mwc
...
2017-12-29T21:20:56.800000Z | -53.180, -118.200 | 5.63 Mwc
2017-12-29T23:55:57.800000Z |  -5.560, +150.890 | 5.73 Mwc
To see all events call 'print(CatalogObject.__str__(print_all=True))'


In [ ]:
from obspy.clients.fdsn.client import Client
from obspy.geodetics.base import gps2dist_azimuth
from sph_loc import sph_loc

import pandas as pd

!rm 'ssbpts.csv'

def save_dataframe(dataframe, file = 'ssbpts.csv', header = False):
    dataframe.to_csv(file, index = False, mode = 'a', header = header)
    
def init_dataframe():
    dataframe = pd.DataFrame(columns =
                      ['Event', 'Network', 'Station', 'Range', 'Azimuth',
                       'BAzimuth', 'EvtLat','EvtLon', 'StaLat', 'StaLon', 'MidLat','MidLon'])
    
    return dataframe

df = init_dataframe()
    
save_dataframe(df, header = True)
    
irow = 0

client = Client('IRIS')

for evt in cat:

    etag = evt.event_descriptions[1].text
    
    etime = evt.preferred_origin().time
    elat  = evt.preferred_origin().latitude
    elon  = evt.preferred_origin().longitude
    
    try:
        inv = client.get_stations(network='*', station='*', channel='LH?', level='station',
                                  startbefore = etime, endafter = etime+60*60,
                                 includerestricted = False, includeavailability=True)
    except:
        print('Request Error')
        continue
        
    for net in inv:
        for sta in net:
            if sta.restricted_status != 'open':
                continue
            
            slat = sta.latitude
            slon = sta.longitude
            
            delm, az, baz = gps2dist_azimuth(elat, elon, slat, slon)
            deldeg = delm/1000./111.11
            if deldeg > 55:
                mlat, mlon = sph_loc(elat,elon,deldeg/2.,az)
                
                if mlon > 180.:
                    mlon = mlon - 360.
                
                df.loc[irow] = [etag, net.code, sta.code, deldeg, az, baz, elat, elon, slat, slon, mlat, mlon]
                irow += 1
                if irow % 1000 == 0:  #write and drop memory load
                    save_dataframe(df)
                    df = init_dataframe()

save_dataframe(df)

rm: ssbpts.csv: No such file or directory
